In [1]:
import utils
import processing as pr

In [2]:
import math
import numpy as np

In [3]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow import keras

In [4]:
class PositionalEmbedding(layers.Layer):
    def __init__(self, d_model, max_len = 5000):
        super(PositionalEmbedding, self).__init__()
        pe = tf.Variable(tf.zeros([max_len,d_model], tf.float32), trainable=False) # pe = (max_len,d_model)
        position = tf.range(0,max_len,dtype=tf.float32)[:, tf.newaxis] # position = (max_len,1)
        div_term = tf.math.exp(tf.range(0,d_model,2,tf.float32)*-(tf.math.log(10000.0)/d_model)) # div_term = (d_model/2,)

        pe = pe[:,0::2].assign(tf.math.sin(position*div_term))
        pe = pe[:,1::2].assign(tf.math.cos(position*div_term))

        self.pe = pe[tf.newaxis,:] # pe = (1,max_len,d_model)

    def call(self, x):
        # inputs = (batch_size, seqs, channels)
        return self.pe[:,:tf.shape(x)[1]] # (1,seqs,d_model)

In [5]:
class TokenEmbedding(layers.Layer):
    def __init__(self, d_model):
        super(TokenEmbedding, self).__init__()
        self.tokenConv = tf.keras.layers.Conv1D(filters=d_model, kernel_size=3, padding='valid', use_bias=False, kernel_initializer=tf.keras.initializers.HeUniform())

    def call(self, x):
        # x = (batch_size, seqs, channels) 
        x = tf.concat([x[:,-1][:,tf.newaxis], x, x[:,0][:,tf.newaxis]], axis=1) # x = (batch_size, seqs, d_model)
        x = self.tokenConv(x) # x = (batch_size, seqs, d_model)
        return x

In [6]:
class DataEmbedding(layers.Layer):
    def __init__(self,d_model, dropout=0.0):
        super(DataEmbedding,self).__init__()

        self.value_embedding = TokenEmbedding(d_model=d_model)
        self.position_embedding = PositionalEmbedding(d_model=d_model)

        self.dropout = tf.keras.layers.Dropout(rate = dropout)

    def call(self, x):
        # inputs = (batch_size, seqs, channels)
        x = self.value_embedding(x) + self.position_embedding(x) # x = (batch_size, seqs, d_model)
        return self.dropout(x) 

In [7]:
class TriangularCasualMask():
    def __init__(self, B, L):
        mask_shape = [B, 1, L, L]
        self._mask = tf.experimental.numpy.triu(tf.ones(mask_shape), k=1)
    
    @property
    def mask(self):
        return tf.cast(self._mask, tf.bool)

In [8]:
class AnomalyAttention(layers.Layer):
    def __init__(self, win_size, mask_flag=True, scale=None, attention_dropout=0.0, output_attention=False):
        super(AnomalyAttention, self).__init__()
        self.scale = scale
        self.mask_flag = mask_flag
        self.output_attention = output_attention
        self.dropout = tf.keras.layers.Dropout(rate = attention_dropout)
        window_size = win_size
        distances = tf.Variable(tf.zeros([window_size,window_size]), trainable=False)
        for i in range(window_size):
            for j in range(window_size):
                distances = distances[i,j].assign(abs(i-j)) # distances = (seqs, seqs)
        self.distances = tf.constant(distances)
    
    def call(self, queries, keys, values, sigma, attn_mask):
        # queries, keys, values = (batch_size, seqs, n_heads, d_keys), sigma = (batch_size, seqs, n_heads)
        B, L, H, E = queries.shape # L=seqs, H=n_heads, E=d_keys
        _, S, _, D = values.shape # S=seqs, D=d_keys
        scale = self.scale or 1./ math.sqrt(E)

        scores = tf.einsum("blhe,bshe->bhls", queries, keys) # scores = (batch_size, n_heads, seqs, seqs)
        if self.mask_flag:
            if attn_mask is None:
                attn_mask = TriangularCasualMask(B, L)
            scores = tf.where(attn_mask.mask, -np.inf, scores)
        attn = scale * scores # attn = (batch_size, n_heads, seqs, seqs)

        sigma = tf.transpose(sigma, perm=[0,2,1]) # sigma = (batch_size, n_heads, seqs)
        window_size = tf.shape(attn)[-1] # window_size = seqs
        sigma = tf.math.sigmoid(sigma*5) + 1e-5
        sigma = tf.math.pow(3,sigma) - 1
        sigma = tf.tile(sigma[...,tf.newaxis],[1,1,1,window_size]) # sigma = (batch_size, n_heads, seqs, seqs)
        prior = tf.tile(self.distances[tf.newaxis,...][tf.newaxis,...], [sigma.shape[0],sigma.shape[1],1,1]) # prior = (batch_size, n_heads, seqs, seqs)
        prior = 1.0 / (math.sqrt(2*math.pi)*sigma)*tf.exp(-prior**2/2/(sigma**2)) # (batch_size, n_heads, seqs, seqs)

        series = self.dropout(tf.nn.softmax(attn, axis=-1)) # series = (n_heads, seqs,seqs)
        V = tf.einsum("bhls,bshd->blhd", series, values) # V = (batch_size, seqs, n_heads, d_keys)

        if self.output_attention:
            return (V, series, prior, sigma)
        else:
            return (V, None)

In [9]:
class AttentionLayer(layers.Layer):
    def __init__(self, attention, d_model, n_heads, d_keys=None, d_values=None):
        super(AttentionLayer, self).__init__()

        d_keys = d_keys or (d_model // n_heads)
        d_values = d_values or (d_model//n_heads)
        self.inner_attention = attention
        self.query_projection = tf.keras.layers.Dense(d_keys*n_heads)
        self.key_projection = tf.keras.layers.Dense(d_keys*n_heads)
        self.value_projection = tf.keras.layers.Dense(d_values*n_heads)
        self.sigma_projection = tf.keras.layers.Dense(n_heads)
        self.out_projection = tf.keras.layers.Dense(d_model)

        self.n_heads = n_heads

    def call(self, queries, keys, values, attn_mask):
        # queries = keys = values = (batch_size, seqs, d_model)
        B, L, _ = queries.shape # L = seqs
        _, S, _ = keys.shape # S = seqs
        H = self.n_heads
        x = queries # x = (batch_size, seqs, d_model)
        queries = tf.reshape(self.query_projection(queries),[B,L,H,-1]) # (batch_size, seqs, d_model) -> (batch_size, seqs, n_heads, d_keys)
        keys = tf.reshape(self.key_projection(keys), [B,S,H,-1]) # (batch_size, seqs, d_model) -> (batch_size, seqs, n_heads, d_keys)
        values = tf.reshape(self.value_projection(values), [B, S,H,-1]) # (batch_size, seqs, d_model) -> (batch_size, seqs, n_heads, d_keys)
        sigma = tf.reshape(self.sigma_projection(x), [B,L,H]) # (batch_size, seqs, n_heads) -> (batch_size, seqs, n_heads)

        out, series, prior, sigma = self.inner_attention(queries, keys, values, sigma, attn_mask) # out=(seqs,n_heads,d_keys) series,prior,sigma=(n_heads,seqs,seqs)
        out = tf.reshape(out,[B, L,-1]) # out = (batch_size, seqs, d_model)

        return self.out_projection(out), series, prior, sigma

In [10]:
class EncoderLayer(layers.Layer):
    def __init__(self, attention, d_model, d_ff=None, dropout=0.1, activation='relu'):
        super(EncoderLayer, self).__init__()
        d_ff = d_ff or 4*d_model
        self.attention = attention
        self.conv1 = tf.keras.layers.Conv1D(d_ff, kernel_size=1)
        self.conv2 = tf.keras.layers.Conv1D(d_model, kernel_size=1)
        self.norm1 = keras.layers.LayerNormalization()
        self.norm2 = keras.layers.LayerNormalization()
        self.dropout = tf.keras.layers.Dropout(dropout)
        self.activation = tf.nn.relu if activation == 'relu' else tf.nn.gelu

    def call(self, x, attn_mask=None):
        # x = (batch_size, seqs, d_model)
        new_x, attn, mask, sigma = self.attention(x,x,x,attn_mask=attn_mask) # new_x=(batch_size,seqs,d_model) attn=(batch_size,n_heads,seqs,seqs) mask=(batch_size,n_heads,seqs,seqs) sigma=(batch_size,n_heads,seqs,seqs)
        x = x + self.dropout(new_x) # x = (batch_size, seqs, d_model)
        y = x = self.norm1(x) # y = (batch_size, seqs, d_model)
        y = self.dropout(self.activation(self.conv1(y))) # y = (batch_size, seqs, d_model)
        y = self.dropout(self.conv2(y)) # y = (batch_size, seqs, d_model)

        return self.norm2(x+y), attn, mask, sigma

In [11]:
class Encoder(layers.Layer):
    def __init__(self, attn_layers, norm_layer=None):
        super(Encoder, self).__init__()
        self.norm = norm_layer
        self.attn_layers = attn_layers

    def call(self, x, attn_mask=None):
        # x = (batch_size, seqs, d_model)
        series_list = list()
        prior_list = list()
        sigma_list = list()
        for attn_layer in self.attn_layers.layers:
            x, series, prior, sigma = attn_layer(x, attn_mask=attn_mask)
            series_list.append(series)
            prior_list.append(prior)
            sigma_list.append(sigma)

        if self.norm is not None:
            x = self.norm(x)
        
        return x, series_list, prior_list, sigma_list

In [12]:
class AnomalyTransformer(tf.keras.Model):
    def __init__(self, win_size, c_out, d_model=512, n_heads=8, e_layers=3, d_ff=512, dropout=0.0, activation='gelu', output_attention=True):
        super(AnomalyTransformer, self).__init__()
        self.output_attention = output_attention

        self.embedding = DataEmbedding(d_model, dropout)

        self.encoder = Encoder(
            [
                EncoderLayer(
                    AttentionLayer(
                        AnomalyAttention(win_size, False, attention_dropout=dropout,output_attention=output_attention), 
                        d_model, n_heads),
                    d_model, 
                    d_ff, 
                    dropout=dropout, 
                    activation=activation
                ) for l in range(e_layers)
            ], 
            norm_layer=tf.keras.layers.LayerNormalization()
        )

        self.projection = tf.keras.layers.Dense(c_out)
    
    def call(self,x):
        x = self.embedding(x)
        x, series, prior, sigmas = self.encoder(x)
        x = self.projection(x)

        if self.output_attention:
            return x, series, prior, sigmas
        else:
            return x

In [13]:
def kl_loss(p,q):
    res = p*(tf.math.log(p+0.0001) - tf.math.log(q+0.0001))
    return tf.reduce_mean(tf.reduce_sum(res, axis=-1), axis=1)

In [14]:
mse = tf.keras.losses.MeanSquaredError()
optimizer = tf.keras.optimizers.Adam(0.0001)

In [15]:
dataloader = pr.data_loader()

In [16]:
train_set, test_set, label_set = dataloader.SMD()

In [17]:
train_set, test_set = dataloader.min_max(train_set, test_set)
train_set_overlap = dataloader.window_overlap(train_set,100,0)
test_set_overlap = dataloader.window_overlap(test_set,100,0)

In [18]:
train_set_overlap.shape

(4968, 100, 51)

In [19]:
model = AnomalyTransformer(win_size=train_set_overlap.shape[1],c_out=train_set_overlap.shape[-1], e_layers=3)

In [20]:
epochs = 10
batch_size = 256

In [21]:
from tqdm import tqdm
from IPython import display

In [22]:
for epoch in range(epochs):
    print("\nStart of epoch %d" % (epoch,))
    copy_set = np.copy(train_set_overlap)
    for k in tqdm(range(copy_set.shape[0]//batch_size)):
        loss1_list = list()
        rec_loss_list = list()
        prior_loss_list = list()
        batch_mask = np.random.choice(copy_set.shape[0], batch_size, replace=False)
        batch = copy_set[batch_mask]
        series_loss = 0.0
        prior_loss = 0.0
        with tf.GradientTape(persistent=True) as tape:
            input = tf.constant(batch, dtype='float32')
            output, series, prior, _ = model(input)
            # output = model(input)
            for u in range(len(prior)):
                series_loss += tf.reduce_mean(kl_loss(series[u],
                tf.stop_gradient((prior[u] / tf.tile(tf.reduce_sum(prior[u], axis=-1)[...,tf.newaxis], [1,1,1,100]))))) \
                + tf.reduce_mean(kl_loss(tf.stop_gradient((prior[u] / tf.tile(tf.reduce_sum(prior[u], axis=-1)[...,tf.newaxis], [1,1,1,100]))), series[u]))
                prior_loss += tf.reduce_mean(kl_loss(tf.stop_gradient(series[u]), \
                (prior[u] / tf.tile(tf.reduce_sum(prior[u], axis=-1)[...,tf.newaxis], [1,1,1,100])))) \
                + tf.reduce_mean(kl_loss((prior[u] / tf.tile(tf.reduce_sum(prior[u], axis=-1)[...,tf.newaxis], [1,1,1,100])), tf.stop_gradient(series[u])))
            series_loss = series_loss/len(series)
            prior_loss = prior_loss/len(prior)
            rec_loss = mse(output, input)
            loss1 = rec_loss - 3*series_loss
            loss2 = rec_loss + 3*prior_loss
            loss1_list.append(loss1.numpy())
            rec_loss_list.append(rec_loss.numpy())
            prior_loss_list.append(prior_loss.numpy())
        gradients1 = tape.gradient(loss1, model.trainable_variables)
        gradients2 = tape.gradient(loss2, model.trainable_variables)
        optimizer.apply_gradients((grad,var) for (grad,var) in zip(gradients1, model.trainable_variables) if grad is not None)
        optimizer.apply_gradients((grad,var) for (grad,var) in zip(gradients2, model.trainable_variables) if grad is not None)
        copy_set = np.delete(copy_set,batch_mask,axis=0)
    display.clear_output()
    print("loss1:", np.average(loss1_list))
    print("rec:", np.average(rec_loss_list))
    print("prior:", np.average(prior_loss_list))

loss1: -46.213966
rec: 0.075172015
prior: 15.429713


In [23]:
def mse_test(x,y):
    return (x-y)**2

In [24]:
attens_energy = list()
test_batch_size = 100
num = test_set_overlap.shape[0] // test_batch_size
for i in range(num):
    test_input = test_set_overlap[i*100:(i+1)*100]
    test_output, test_series, test_prior, _ = model(test_input)
    test_loss = tf.reduce_mean(mse_test(test_input, test_output), axis=-1)
    for u in range(len(test_prior)):
        if u == 0:
            test_series_loss = kl_loss(test_series[u], (test_prior[u] / tf.tile(tf.reduce_sum(test_prior[u], axis=-1)[...,tf.newaxis], [1,1,1,100])))*50
            test_prior_loss = kl_loss(test_prior[u] / tf.tile(tf.reduce_sum(test_prior[u], axis=-1)[...,tf.newaxis], [1,1,1,100]), test_series[u])*50
        else:
            test_series_loss += kl_loss(test_series[u], (test_prior[u] / tf.tile(tf.reduce_sum(test_prior[u], axis=-1)[...,tf.newaxis], [1,1,1,100])))*50
            test_prior_loss += kl_loss(test_prior[u] / tf.tile(tf.reduce_sum(test_prior[u], axis=-1)[...,tf.newaxis], [1,1,1,100]), test_series[u])*50
    metric = tf.nn.softmax((-test_prior_loss-test_series_loss), axis=-1)
    cri = metric*test_loss
    attens_energy.append(cri)

In [25]:
attens_energy = np.concatenate(attens_energy, axis=0).reshape(-1)

In [26]:
from sklearn import metrics
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve

In [27]:
sort_score = np.sort(attens_energy)

In [28]:
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import accuracy_score

In [29]:
best_f = 0
best_precision = 0
best_recall = 0
best_thr = 0
gt = label_set
for i in range(0,100):
    thr = i/500
    pred = (attens_energy > thr).astype(int)
    anomaly_state = False
    for i in range(len(pred)):
        if gt[i] == 1 and pred[i] == 1 and not anomaly_state:
            anomaly_state = True
            for j in range(i, 0, -1):
                if gt[j] == 0:
                    break
                else:
                    if pred[j] == 0:
                        pred[j] = 1
            for j in range(i, len(gt)):
                if gt[j] == 0:
                    break
                else:
                    if pred[j] == 0:
                        pred[j] = 1
        elif gt[i] == 0:
            anomaly_state = False
        if anomaly_state:
            pred[i] = 1
    precision, recall, f_score, support = precision_recall_fscore_support(label_set[:attens_energy.shape[0]],pred, average='binary')
    if f_score > best_f:
        best_f = f_score
        best_precision = precision
        best_recall = recall
        best_thr = thr

In [30]:
print("precision:", best_precision)
print("recall:", best_recall)
print("f_score:", best_f)
print("thr:", best_thr)

precision: 0.9342781442822341
recall: 0.9926319170240502
f_score: 0.9625714495090135
thr: 0.02
